Reset everything

In [0]:
!kill -9 -1

# Building darknet


> most of the code is from https://github.com/pavisj/YoloV3_video_colab


Clone darknet improved repository

In [0]:
!git clone https://github.com/AlexeyAB/darknet/

Go to darknet folder

In [0]:
cd darknet

Install CUDA for GPU support

In [0]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

In [0]:
!apt update

In [0]:
!apt install cuda-8-0 -y

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [0]:
!apt install gcc-5 g++-5 -y

In [0]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

Activate OpenCV and GPU in Makefile

In [0]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile

Install OpenCV

In [0]:
!apt-get install libopencv-dev

Make darknet

In [0]:
!make

Download yolov3 weights

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

Function to show the predictions.jpg

In [0]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def showpred():
  image = cv2.imread("predictions.jpg")
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Test darknet with dog.jpg

In [0]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

In [0]:
showpred()

## Prepare darknet for custom training
Upload in darknet folder:
- coins.data
- coins.names
- coins-yolov3.cfg
- coins-train.txt
- coins-test.txt
- coins.zip
- (coins-yolov3_last.wights)


In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': ''})
download.GetContentFile('coins.zip')

In [0]:
!unzip coins

In [0]:
download = drive.CreateFile({'id': ''})
download.GetContentFile('coins-yolov3_x000.weights')

Start training with darknet53.conv.74 weights

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [0]:
!./darknet detector train coins.data coins-yolov3.cfg darknet53.conv.74 -dont_show -map | tee log.txt

or start training with last weights

In [0]:
!./darknet detector train coins.data coins-yolov3.cfg coins-yolov3_1000.weights -dont_show -map | tee log.txt

Upload weights to google drive

In [0]:
for i in [3,4,5]:
  upload = drive.CreateFile({'title': 'coins-yolov3_' + str(i) + 'x000.weights'})
  upload.SetContentFile('backup/coins-yolov3_' + str(i) + 'x000.weights')
  upload.Upload()